In [1]:
import os
import re
import json
import traceback
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

from kiwipiepy import Kiwi ## 한글 형태소 분석기

from langchain.schema import Document
from langchain.vectorstores import FAISS  ## 벡터 DB
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

eval_data = load_jsonl("../dataset/eval.jsonl")
print(len(eval_data))
print(eval_data[0].keys())

doc_data = load_jsonl("../dataset/documents.jsonl")
print(len(doc_data))
print(doc_data[0].keys())

220
dict_keys(['eval_id', 'msg'])
4272
dict_keys(['docid', 'src', 'content'])


In [3]:
# 문서 로드
raw_documents = load_jsonl('../dataset/documents.jsonl')

# 문서에서 docid를 포함한 Document 리스트 생성
documents = []
for doc in raw_documents:
    doc_id = doc['docid']  # JSONL 파일에서 docid 추출
    content = doc['content']  # 문서 내용 추출
    documents.append(Document(page_content=content, metadata={"doc_id": doc_id}))

In [4]:
kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [5]:
# BM25와 FAISS 기반 검색기 생성
bm25 = BM25Retriever.from_documents(documents)
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

# FAISS 벡터 DB 생성 및 메타데이터와 함께 저장
faiss = FAISS.from_documents(documents, OpenAIEmbeddings()).as_retriever()

bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [6]:
retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

In [7]:
# 검색 결과 출력 함수
def print_search_results(retrievers, query):
    print(f"Query : {query}")
    for name, retriever in retrievers.items():
        results = retriever.invoke(query)
        print(f"{name} 결과:")
        for result in results:
            print(f"문서 ID: {result.metadata['doc_id']} 내용: {result.page_content}")
        print("===" * 20)

In [8]:
print(print_search_results(retrievers, "ktx의 평균 속도는 얼마인가?"))

Query : ktx의 평균 속도는 얼마인가?
bm25 결과:
문서 ID: e8d07278-c590-43e2-b99a-47d5adaf123d 내용: 아르투로는 3,000미터 경주에서 10분이라는 기록으로 결승점에 도달했습니다. 이를 통해 우리는 아르투로의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 걸린 시간으로 나눈 값입니다. 따라서, 아르투로의 평균 속도는 3,000미터를 10분으로 나눈 값인 300미터/분입니다. 이를 미터/초로 변환하면 5 m/s가 됩니다. 따라서, 아르투로의 평균 속도는 5 m/s입니다.
문서 ID: 8ca2ebf5-9f0d-4d2f-bfef-92b44c920d2e 내용: 고무 오리 경주에서 우승한 오리는 10분 동안 300미터를 이동했습니다. 이를 통해 우승한 오리의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 소요된 시간으로 나눈 값입니다. 따라서, 이 경우 오리의 평균 속도는 300미터를 10분으로 나눈 값인 30미터/분입니다. 그러나, 문제에서 속도를 미터/초로 표현하라고 하였으므로, 분 단위를 초 단위로 변환해야 합니다. 1분은 60초이므로, 30미터/분은 0.5미터/초입니다. 따라서, 고무 오리 경주에서 우승한 오리의 평균 속도는 0.5미터/초입니다.
문서 ID: 525f7b96-8422-4e57-b54b-32aff1d42aed 내용: 학생들은 학교 운동장에서 거북이를 관찰하였습니다. 그들은 거북이가 30분 동안 이동한 거리를 측정하였습니다. 거북이가 이 시간 동안 40미터를 걸었다면, 평균 속도는 얼마였을까요? 이 문제를 풀기 위해서는 거리를 시간으로 나누어야 합니다. 따라서, 거북이의 평균 속도는 40미터를 30분으로 나눈 값인 1.33m/min입니다. 그러나, 답안에서는 속도를 시간당 단위로 표기하였습니다. 따라서, 거북이의 평균 속도는 1.33m/min을 시간당 단위로 변환한 80m/h입니다.
문서 ID: fe6a254b-0f86-4025-b867-b35d56273c8d 

In [ ]:
client = OpenAI()
model = "gpt-4o"

In [ ]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    return cleaned_response

In [ ]:
standalone_content = ("입력된 내용이 한 줄의 문장인지 여러 줄의 대화 내용인지 분류하세요. 반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다."
                      "- 단일 문장인 경우 : {\"multi_turn\": false, \"query\": \"입력 문장\"} "
                      "- 여러 줄의 대화 내용인 경우 : {\"multi_turn\": true, \"query\": \"대화 내용을 종합하여 만든 새로운 질문\"}")


def create_standalone_query(query):
    # query가 중첩된 리스트 형태일 때 텍스트를 추출
    if isinstance(query, list) and 'content' in query[0]:
        query = query[0]['content'][0]['content']
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": standalone_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response


In [ ]:
domain_content = (
    "입력된 텍스트가 상식에 대한 질문인지 아니면 단순한 일상적인 대화인지 구분하세요."
    " - 상식에 대한 질문이란, 사용자가 지식이나 정보를 얻기 위해 하는 질문입니다. 예: '나무의 분류에 대해 조사하는 방법은?', 'Dmitri Ivanovsky가 누구야?', '남녀 관계에서 정서적인 행동이 왜 중요해?'"
    " - 일상적인 대화란, 주로 감정이나 의견을 표현하거나 대화의 흐름을 유지하기 위한 내용입니다. 예: '요새 너무 힘들다.', '니가 대답을 잘해줘서 너무 신나!', '이제 그만 얘기해!', '오늘 너무 즐거웠다!', ''너는 누구니?', '너는 어떤 능력을 가지고 있니?"
    " 반드시 JSON 형식으로 응답하세요. 키와 값은 모두 이중 따옴표로 감싸야 합니다."
    " - 상식에 대한 질문인 경우: {\"out_of_domain\": false, \"query\": \"입력된 쿼리를 그대로 반환\"}"
    " - 일상적인 대화인 경우: {\"out_of_domain\": true, \"query\": \"적절한 응답을 할 수 없습니다.\"}"
)


def domain_check(query):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": domain_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [ ]:
func_call_prompt = """
## Role : 과학 상식 전문가

## Instructions
입력된 질문에 적합한 정보를 검색하기 위해 search api를 호출할 수 있어야 한다.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query for information searching."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]

qa_prompt = """

## Role : 과학 상식 전문가

## Instructions
- 입력 받은 질문과 reference 정보들을 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
"""

def answer_question(messages):
    # 함수의 출력값 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 쿼리 검사1: 멀티턴 대화인 경우 standalone query를 생성한다.
    result1 = create_standalone_query(messages)

    # 쿼리 검사2: 쿼리가 과학 상식과 관련된 것인지 검사한다.
    result2 = domain_check(result1['query'])

    if not result2['out_of_domain']:
        query = result2['query']
        response['standalone_query'] = query

        # kiwi_bm25_faiss_73 retriever를 사용하여 검색 수행
        search_result = retrievers["kiwi_bm25_faiss_73"].invoke(query)

        retrieved_context = []
        for doc in search_result:
            retrieved_context.append(doc.page_content)
            response["topk"].append(doc.metadata.get('docid'))
            response["references"].append({
                "score": doc.metadata.get('score'),
                "content": doc.page_content
            })

        # 검색 결과를 assistant 메시지에 추가
        content = json.dumps(retrieved_context)
        # 여기서 content는 문자열로 저장되도록 수정
        messages.append({"role": "assistant", "content": content})

        # 검색된 문서들을 바탕으로 최종 답변 생성
        msg = [{"role": "system", "content": qa_prompt}] + messages  # 올바른 구조 확인
        try:
            qaresult = client.chat.completions.create(
                model=model,
                messages=msg,
                temperature=0,
                seed=1,
                timeout=30
            )
            response["answer"] = qaresult.choices[0].message.content

        except Exception as e:
            traceback.print_exc()
            return response

    else:
        response["answer"] = "질문이 과학 상식에 해당하지 않습니다."

    return response


In [ ]:
# 평가를 위한 함수 정의 및 실행
def eval_rag(eval_filename, output_filename):
    with open(eval_filename, 'r', encoding='utf-8') as f, open(output_filename, "w", encoding='utf-8') as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'[Test {idx:>04}] | Question: {j["msg"]}')

            messages = [{"role": "user", "content": j["msg"]}]
            print(messages)

            result = answer_question(messages)
            print(result)

            break

In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../dataset/eval.jsonl", "../dataset/sample_submission.csv")